<a href="https://colab.research.google.com/github/Berigny/AI-Entrainment-Protocol/blob/main/LENR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low Energy Nuclear (LENR) Cycles Model
This notebook tests whether a minimal, topology-first model can simulate and falsify a low-energy nuclear reaction (LENR) cycle driven by breath-like EM coherence. Using Discrete Exterior Calculus (DEC) on a star-tetrahedral complex with a shared centroid, we evolve fields, enforce first-law energy accounting, monitor entropy production and phase relations, and—optionally—add an adelic (ℝ × ℚ_p) layer to represent hierarchical memory at the fusion node. If quantised heat bursts do not emerge with lawful topology, pre-burst EM signatures, rising EPR, and correct energy balance, the hypothesis is weakened. If they do, we proceed to richer modelling and lab validation. The prize is significant: a path to low-cost, high-yield, sustainable energy in an energy-hungry world.

## Falsification criteria

We consider the LENR cycle unsupported if any lawful configuration fails to produce:
(i) pre-burst EM coherence with ~90° E–M phase and centroid mediation,
(ii) quantised heat steps at the sink,
(iii) a non-negative EPR that rises prior to bursts, and
(iv) first-law integrity (residual ≈ 0).
Controls must behave as expected: illegal shortcuts reduce irreversibility or break energy accounting; removing C→3 or 1→2 kills bursts; random primes revert to baseline.

## Notebook Overview & Research Aim
### Goal
- Simulate and *attempt to falsify* LENR-like cycles on a tetrahedral (and star-tetrahedral) topology.
- If not falsified, motivate deeper modelling and lab tests for low-cost, high-yield, sustainable energy.
### Key Ideas
- DEC on simplicial complexes; mediation via centroid **C**; breath-driven, asymmetric flows; adelic (ℝ×ℚ_p) layer.

In [ ]:
# @title 1. **Imports & Global Config** { display-mode: "form" }
# @markdown **What this sets**
# @markdown - Imports, numeric precision/warnings, plotting defaults, reproducibility seeds.

import numpy as np
import networkx as nx
from mpmath import mpf, power
import matplotlib.pyplot as plt


# Geometry (simplices, orientations, dual volumes)

In [ ]:
# @title 2. **Geometry — S1 (Single Tetra) and Star (S1+S2) with Cubic Bridges** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Builds the base directed graph `G` for one tetrahedron (S1) or the full star (S1+S2) if `USE_STAR=True`.
# @markdown - Defines vertices `{0,1,2,3,C}` for S1 and adds `{4,5,6,7}` for S2 in star mode.
# @markdown - Encodes work-like edges, heat-dump edges, and mediation paths through centroid `C`.
# @markdown - Adds optional cross-tetra “cubic” bridges for coherent work transfer and heat redistribution.
# @markdown - Inserts self-loops for stability/linger dynamics.
# @markdown - Labels canonical edge sets (`work_edges`, `heat_edges`) inside `G.graph` for later diagnostics.

# === Geometry — S1 (tetra) and optional Star (S1+S2) with cubic bridges ===
# Inputs: USE_STAR (bool) defined in C01_config; default to False if missing
try:
    USE_STAR
except NameError:
    USE_STAR = False

import networkx as nx
from mpmath import mpf

# Nodes
S1 = ['0','1','2','3']      # sinks even / branches odd
S2 = ['4','5','6','7']
ALL = (S1 + ['C']) if not USE_STAR else (S1 + S2 + ['C'])

# Helper
r = lambda x: mpf(str(x))

# Core single-shell flows (work/heat + mediation) on S1
core_edges_S1 = [
    ('C','1', {'rate': r(0.5)}),     # pump
    ('1','2', {'rate': r(0.8)}),     # E→M (work)
    ('2','C', {'rate': r(0.9)}),     # return to mediator
    ('C','3', {'rate': r(0.7)}),     # squeeze/fusion path
    ('3','0', {'rate': r(1.0)}),     # heat dump
    ('0','C', {'rate': r(0.3)}),     # reset
]

# Optional second shell (S2)
core_edges_S2 = [
    ('C','5', {'rate': r(0.5)}),
    ('5','6', {'rate': r(0.8)}),
    ('6','C', {'rate': r(0.9)}),
    ('C','7', {'rate': r(0.7)}),
    ('7','4', {'rate': r(1.0)}),
    ('4','C', {'rate': r(0.3)}),
]

# Cross-shell bridges (only used if USE_STAR)
cross_edges = [
    ('3','6', {'rate': r(0.35)}),  # work bridge S1→S2
    ('7','2', {'rate': r(0.25)}),  # heat-biased return to S1 magnetic
    ('5','0', {'rate': r(0.20)}),  # heat assist into S1 sink
    ('1','6', {'rate': r(0.20)}),  # optional E→M assist across shells
]

# Build graph once
G = nx.DiGraph()
G.add_nodes_from(ALL)
G.add_edges_from(core_edges_S1)

if USE_STAR:
    G.add_edges_from(core_edges_S2 + cross_edges)
    self_loop_nodes = S1 + S2
else:
    self_loop_nodes = S1

# Self-loops (linger/stability)
G.add_edges_from([(n, n, {'rate': r(0.4)}) for n in self_loop_nodes])

# Label canonical edge sets (for later coupling/diagnostics)
work_edges = {('1','2'), ('3','6')} if USE_STAR else {('1','2')}
heat_edges = {('3','0')} | ({('7','2'), ('5','0')} if USE_STAR else set())

G.graph['work_edges'] = work_edges
G.graph['heat_edges'] = heat_edges

print(f"Graph ready. USE_STAR={USE_STAR} |V|={G.number_of_nodes()} |E|={G.number_of_edges()}")


Graph ready. USE_STAR=False |V|=5 |E|=10


In [ ]:
# @title 3. **Breath Operator — Inhale/Exhale Rhythm** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Defines a transformer `apply_breath(G_base, step)` that scales edge rates
# @markdown   according to the breath cycle.
# @markdown - Even steps = inhale: favour flows into sinks (`0,4`) and mediator `C`.
# @markdown - Odd steps = exhale: favour flows out of `C` toward pump branches (`1,3,5,7`).
# @markdown - Parameters `inhale_gain` and `exhale_gain` control modulation strength.
# @markdown - Returns a fresh graph copy and the current phase label (`"in"` / `"out"`).


def apply_breath(G_base, step, inhale_gain=1.15, exhale_gain=1.10,
                 sinks=('0','4'), sources=('C',), pumps=('1','3','5','7')):
    """Scale edge rates by breath phase. Even steps = inhale (toward sinks/C), odd = exhale (from C→pumps)."""
    phase = 'in' if (step % 2) == 0 else 'out'
    G = G_base.copy()
    for u, v, d in G.edges(data=True):
        r = float(d.get('rate', 0.0))
        if phase == 'in' and (v in sinks or v in sources):
            d['rate'] = r * inhale_gain
        elif phase == 'out' and (u in sources and v in pumps):
            d['rate'] = r * exhale_gain
    return G, phase


# DEC operators (∂, Hodge stars, audits)

In [ ]:
# @title 4. **DEC Backbone — ∂ Operators & Diagonal Hodge Stars (with Audits)** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Builds the simplicial backbone (independent of the Markov graph): vertices, oriented edges, faces.
# @markdown - Constructs boundary maps: **B₁** (nodes→edges) and **B₂** (edges→faces).
# @markdown - Defines **diagonal SPD Hodge stars** `Star1` (edges×edges) and `Star2` (faces×faces).
# @markdown - Runs audits: `||B₁·B₂||∞ ≈ 0`, ranks of B₁/B₂, SPD minima; exposes `energy_field(E,B)`.
# @markdown **Notes**
# @markdown - No rectangular/“incidence-averaged” Hodge stand-ins here. Those are deprecated.

import numpy as np
from scipy.sparse import csr_matrix

# ---- 1) Vertex & face set (S1 always; S2 included if USE_STAR=True) ----
try:
    USE_STAR
except NameError:
    USE_STAR = False

# Vertices: S1 plus shared centroid C; extend with S2 in star mode
V = ['0','1','2','3','C'] + (['4','5','6','7'] if USE_STAR else [])

# Faces:
# - S1 outer + centroid faces
F_S1_outer = [('0','1','2'), ('0','1','3'), ('0','2','3'), ('1','2','3')]
F_S1_cent  = [('0','1','C'), ('0','2','C'), ('0','3','C'), ('1','2','C'), ('1','3','C'), ('2','3','C')]

# - Optional S2 outer + centroid faces (only if USE_STAR)
F_S2_outer = [('4','5','6'), ('4','5','7'), ('4','6','7'), ('5','6','7')]
F_S2_cent  = [('4','5','C'), ('4','6','C'), ('4','7','C'), ('5','6','C'), ('5','7','C'), ('6','7','C')]

F = F_S1_outer + F_S1_cent + (F_S2_outer + F_S2_cent if USE_STAR else [])

# ---- 2) Oriented edges derived from faces (backbone edges only) ----
edges_backbone = sorted({(a,b) for (a,b,c) in F for (a,b) in ((a,b),(b,c),(c,a))})

# Index maps
node_id = {v:i for i,v in enumerate(V)}
edge_id = {e:i for i,e in enumerate(edges_backbone)}
face_id = {f:i for i,f in enumerate(F)}

# ---- 3) Boundary maps ----
# B1: nodes×edges (∂1)
B1 = np.zeros((len(V), len(edges_backbone)), dtype=float)
for (u,v), ei in edge_id.items():
    B1[node_id[u], ei] = -1.0
    B1[node_id[v], ei] = +1.0
B1 = csr_matrix(B1)

# B2: edges×faces (∂2) with oriented incidence
B2 = np.zeros((len(edges_backbone), len(F)), dtype=float)
for fj,(v0,v1,v2) in enumerate(F):
    for (a,b) in ((v0,v1),(v1,v2),(v2,v0)):
        sign = +1.0
        e = (a,b)
        if e not in edge_id:
            e = (b,a); sign = -1.0
        B2[edge_id[e], fj] += sign
B2 = csr_matrix(B2)

# ---- 4) Diagonal Hodge stars (regularised circumcentric stand-ins) ----
# Use a unit geometric scale here; SI scaling lives in the optional calibration cell.
L = 1.0  # assumed primal edge length scale for now
nE, nF = len(edges_backbone), len(F)

# primal measures
len_e  = np.full(nE, L, dtype=float)                       # |e|
area_f = np.full(nF, (np.sqrt(3)/4.0)*L*L, dtype=float)    # |f| (equilateral)

# crude-but-coherent dual measures for a regular/star tetra
dual_e = np.full(nE, (L*L)/4.0, dtype=float)               # |*e|
dual_f = np.full(nF, (L/3.0), dtype=float)                 # |*f|

star1_diag = dual_e / len_e          # edges→edges
star2_diag = dual_f / area_f         # faces→faces

Star1 = np.diag(star1_diag)          # SPD
Star2 = np.diag(star2_diag)

def star1(x): return Star1 @ x
def star2(x): return Star2 @ x

# ---- 5) Audits ----
D1 = B1.toarray(); D2 = B2.toarray()
B1B2 = D1 @ D2
inf_norm = float(np.max(np.abs(B1B2))) if B1B2.size else 0.0
rank_D1 = int(np.linalg.matrix_rank(D1))
rank_D2 = int(np.linalg.matrix_rank(D2))
is_spd_star1 = bool(np.min(star1_diag) > 0)
is_spd_star2 = bool(np.min(star2_diag) > 0)

print(f"[dims] |V|={len(V)} |E|={nE} |F|={nF}")
print(f"||B1·B2||_∞ = {inf_norm:.3e} (expect 0)")
print(f"rank(B1)={rank_D1}, rank(B2)={rank_D2}")
print(f"Star1 SPD? {is_spd_star1}   Star2 SPD? {is_spd_star2}")
print(f"star1_diag min/max = {star1_diag.min():.3e} / {star1_diag.max():.3e}")
print(f"star2_diag min/max = {star2_diag.min():.3e} / {star2_diag.max():.3e}")

# ---- 6) Energy helper ----
def energy_field(E, B):
    """Discrete EM energy: 0.5*(E⋅(⋆1 E) + B⋅(⋆2 B))."""
    return 0.5*(E @ (Star1 @ E) + B @ (Star2 @ B))


[dims] |V|=5 |E|=15 |F|=10
||B1·B2||_∞ = 0.000e+00 (expect 0)
rank(B1)=4, rank(B2)=10
Star1 SPD? True   Star2 SPD? True
star1_diag min/max = 2.500e-01 / 2.500e-01
star2_diag min/max = 7.698e-01 / 7.698e-01


In [ ]:
# @title 5. **(Optional) SI Calibration — Scales, Materials & CFL Guard** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Sets dimensionful scales (edge length `L_m`), material guesses (ε_r, μ_r), and a conservative CFL-like `global_dt`.
# @markdown - Leaves DEC stars (`Star1/Star2`) untouched; use this only for reporting or SI-coupled experiments.

import numpy as np
from mpmath import mpf

# ---- 1) Geometry scale (metres) ----
L_m = mpf('1e-9')  # 1 nm representative edge length (tune as needed)

# ---- 2) Materials (metal-like defaults) ----
eps0 = mpf('8.854187817e-12')     # F/m
mu0  = mpf('1.25663706212e-6')    # H/m
eps_r = mpf('20')                  # relative permittivity (order 10–100)
mu_r  = mpf('1')                   # non-magnetic
sigma = mpf('1e6')                 # S/m (ballpark for metals; tune/disable if not using ohmic loss)

eps = eps_r * eps0
mu  = mu_r  * mu0
c_si = 1.0 / float(np.sqrt(eps * mu))   # wave speed in medium

# ---- 3) CFL-like time step guard (from Markov rates if available) ----
def cfl_dt_from_rates(graph, safety=0.1):
    if graph is None:
        return None
    try:
        mr = max((float(d.get('rate', 0.0)) for _,_,d in graph.edges(data=True)), default=0.0)
    except Exception:
        mr = 0.0
    if mr <= 0.0:
        return None
    return safety / mr

try:
    G  # if your Markov graph exists, we can use it
    dt_rate = cfl_dt_from_rates(G, safety=0.1)
except NameError:
    dt_rate = None

# geometric CFL (Yee-like): dt ≤ 0.5 * L / c
dt_geom = 0.5 * float(L_m) / float(c_si)

# choose most conservative if both exist
candidates = [x for x in (dt_rate, dt_geom) if x is not None]
global_dt = min(candidates) if candidates else dt_geom

# ---- 4) Export SI globals ----
global_epsilon = float(eps)
global_mu      = float(mu)
global_sigma   = float(sigma)
global_c       = float(c_si)

print(f"L = {float(L_m):.3e} m   eps_r={float(eps_r)}   mu_r={float(mu_r)}   sigma={float(sigma):.2e} S/m")
print(f"c (medium) = {global_c:.3e} m/s")
print(f"dt_geom = {dt_geom:.3e} s   dt_rate = {dt_rate if dt_rate is not None else 'n/a'}")
print(f"=> global_dt = {global_dt:.3e} s")


L = 1.000e-09 m   eps_r=20.0   mu_r=1.0   sigma=1.00e+06 S/m
c (medium) = 6.704e+07 m/s
dt_geom = 7.459e-18 s   dt_rate = 0.1
=> global_dt = 7.459e-18 s


# Time evolution (Maxwell-like DEC loop + optional Markov flux layer)

In [ ]:
# @title 6. **Coupling — Markov → DEC (edge currents & heat sink)** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Provides pure functions to couple the Markov layer to the DEC solver:
# @markdown   • `markov_to_currents(Gk, edge_id, ...) → J` injects **coherent edge currents** on the DEC backbone.
# @markdown   • `accumulate_heat(Gk, dt, ...) → ΔQ` integrates **dissipative heat** from designated heat edges.
# @markdown - Orientation-aware: if a directed Markov edge aligns with a DEC edge, +J; if reversed, −J; if not on the backbone, ignore.
# @markdown - By default reads canonical `work_edges` / `heat_edges` from `G.graph` (set in Geometry). You can also pass explicit sets.

import numpy as np

def _edge_sign_index(u, v, edge_id):
    """Return (sign, idx) for directed pair (u,v) against DEC backbone edge_id map."""
    uv = (u, v)
    vu = (v, u)
    if uv in edge_id:
        return +1.0, edge_id[uv]
    if vu in edge_id:
        return -1.0, edge_id[vu]
    return 0.0, None  # not on backbone

def markov_to_currents(
    Gk,
    edge_id: dict,
    coherence_gain: float = 0.1,
    work_edges: set | None = None,
):
    """
    Map Markov rates to a coherent current vector J on DEC edges.
    Only edges tagged as 'work' contribute (coherent); 'heat' edges are handled in heat accounting.
    """
    J = np.zeros(len(edge_id), dtype=float)
    # pick work set (prefer graph annotation)
    if work_edges is None:
        work_edges = Gk.graph.get('work_edges', set())
    for (u, v, data) in Gk.edges(data=True):
        if (u, v) not in work_edges:
            continue
        rate = float(data.get('rate', 0.0))
        sgn, ei = _edge_sign_index(str(u), str(v), edge_id)
        if ei is None or sgn == 0.0:
            continue
        J[ei] += sgn * coherence_gain * rate
    return J

def accumulate_heat(
    Gk,
    dt: float,
    heat_edges: set | None = None,
):
    """
    Integrate dissipative power from 'heat' edges into a scalar heat sink Q.
    """
    if heat_edges is None:
        heat_edges = Gk.graph.get('heat_edges', set())
    dQ = 0.0
    for (u, v, data) in Gk.edges(data=True):
        if (u, v) in heat_edges:
            dQ += float(data.get('rate', 0.0)) * float(dt)
    return dQ


[dims] |V|=5 |E|=15 |F|=10
||B1·B2||_∞ = 0.000e+00 (expect 0)
rank(B1)=4, rank(B2)=10
Star1 SPD? True   Star2 SPD? True
star1_diag min/max = 2.500e-01 / 2.500e-01
star2_diag min/max = 7.698e-01 / 7.698e-01


In [ ]:
# @title 7. **Core Engine — Leapfrog DEC Loop (Energy & Heat Audit)** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Integrates Maxwell-like fields on the DEC backbone with a **staggered (leapfrog/Yee)** scheme:
# @markdown   • Faraday:   `B^{n+1/2} = B^{n-1/2} - dt * (∂₂ᵀ E^n)`
# @markdown   • Ampère:    `⋆₁ E^{n+1} = ⋆₁ E^n + dt * (∂₁ᵀ ⋆₂ B^{n+1/2} - J^{n+1/2})`
# @markdown - Couples to Markov layer via `apply_breath` (modulates rates) and `markov_to_currents` (injects **J** on edges).
# @markdown - Audits **first-law closure**: logs field energy `U_field(t)`, cumulative heat `Q(t)`, and residual `Δ(U_field+Q)`.
# @markdown
# @markdown **Inputs expected from earlier cells**
# @markdown - DEC: `B1, B2, Star1, Star2, energy_field`, and `edge_id` (from the DEC backbone cell).
# @markdown - Markov: `G` (geometry) and `apply_breath` (breath operator).
# @markdown - Optional SI: `global_dt` (else a safe dt is inferred from rates).
# @markdown
# @markdown **Outputs**
# @markdown - `hist` dict with keys: `heat`, `energy`, `residual`, `phase`, `dt`, `samples`

import numpy as np

def _safe_dt_from_rates(G, default_dt=1e-2, safety=0.1):
    try:
        mr = max((float(d.get('rate', 0.0)) for _, _, d in G.edges(data=True)), default=0.0)
    except Exception:
        mr = 0.0
    if mr > 0.0:
        return safety / mr
    return default_dt

def run_lenr_core(
    G_base,
    steps: int = 400,
    dt: float | None = None,
    coherence_gain: float = 0.1,
    report_every: int = 20,
):
    # ---- Guards (ensure DEC objects exist) ----
    needed = ['B1','B2','Star1','Star2','energy_field','edge_id']
    missing = [k for k in needed if k not in globals()]
    if missing:
        raise RuntimeError(f"Missing DEC objects: {missing}. Run the DEC backbone cell first.")

    # Dense views (small problems → faster)
    D1 = B1.toarray()
    D2 = B2.toarray()
    DT1 = D1.T
    DT2 = D2.T

    # Star1 is diagonal SPD; grab its diagonal for cheap solves
    star1_diag = np.diag(Star1).astype(float)
    star2_mat  = Star2  # diagonal as well
    inv_star1_diag = 1.0 / star1_diag

    # Time step
    if dt is None:
        dt = globals().get('global_dt', None)
    if dt is None:
        dt = _safe_dt_from_rates(G_base, default_dt=1e-2, safety=0.1)
    dt = float(dt)

    # Fields: E on edges (integer time); B on faces (half-step)
    nE = D1.shape[1]
    nF = D2.shape[1]
    E = np.zeros(nE, dtype=float)
    Bf = np.zeros(nF, dtype=float)

    # Heat sink & energy tracker
    Q_sink = 0.0
    U_prev = energy_field(E, Bf)

    hist = {"heat": [], "energy": [], "residual": [], "phase": [], "dt": dt, "samples": []}

    for n in range(steps):
        # Breath-modulated graph & phase tag
        Gk, phase = apply_breath(G_base, n)

        # Coherent current from Markov layer
        J = markov_to_currents(Gk, edge_id=edge_id, coherence_gain=coherence_gain)

        # --- Faraday (half-step for B)
        # B^{n+1/2} = B^{n-1/2} - dt * (∂₂ᵀ E^n)
        Bf = Bf - dt * (DT2 @ E)

        # --- Ampère (integer-step for E) with diagonal ⋆₁ solve
        # ⋆₁ E^{n+1} = ⋆₁ E^n + dt * (∂₁ᵀ ⋆₂ B^{n+1/2} - J)
        drive = DT1 @ (star2_mat @ Bf) - J
        E = E + dt * (drive * inv_star1_diag)

        # Heat accounting
        Q_sink += accumulate_heat(Gk, dt)

        # Energy audit
        U_now = energy_field(E, Bf)
        residual = (U_now + Q_sink) - U_prev
        U_prev = U_now + Q_sink

        # Minimal logging (downsampled)
        if (n % report_every) == 0 or n == steps - 1:
            hist["heat"].append(float(Q_sink))
            hist["energy"].append(float(U_now))
            hist["residual"].append(float(residual))
            hist["phase"].append(phase)
            hist["samples"].append(n)

    return hist

# --- Quick smoke test (comment out if you prefer) ---
# hist = run_lenr_core(G, steps=120, report_every=10)
# print(f"samples={len(hist['samples'])}, heat_last={hist['heat'][-1]:.3e}, residual_last={hist['residual'][-1]:+.3e}")


=== Markov Metrics ===
Centroid reliance R_C     : normal=0.241 | counterfactual=0.230
Electric acceleration ΔE   : normal=0.288 | counterfactual=0.241
Entropy production Σ       : normal=17.347 | counterfactual=14.771
Spectral gap (1-|λ2|)      : normal=0.389 | counterfactual=0.405
Kemeny constant            : normal=4.649 | counterfactual=4.837
Mean hitting time to {0,C} from each node (normal):
                       0: 0.000
                       1: 2.944
                       2: 1.444
                       3: 1.400
                       C: 0.000
Mean hitting time to {0,C} from each node (counterfactual (2→1)):
                       0: 0.000
                       1: 3.222
                       2: 1.722
                       3: 1.400
                       C: 0.000


/tmp/ipython-input-3529139570.py:154: ComplexWarning: Casting complex values to real discards the imaginary part
  return gap, float(K)


# Diagnostics (energy, EPR, phase, visuals)

In [ ]:
# @title Main — Select Pipeline and Geometry { display-mode: "form" }
PIPELINE = "fast"   # "fast" (Cell 6 dec_step) or "si" (Cell 14 leapfrog_step)
GEOMETRY = "S1"     # "S1" or "STAR"

if PIPELINE == "fast":
    print("Running baseline DEC stepper (dimensionless).")
    # This will run the simulation defined in Cell 6
    # The `baseline_history` variable will be available for subsequent cells.
elif PIPELINE == "si":
    print("Running stable SI leapfrog stepper.")
    # This would run the simulation from Cell 14.
    # Note: The `run_star_tetra_cycle` function is defined in Cell 19,
    # so you would need to ensure that cell is run before this one if using the "si" pipeline.
else:
    raise ValueError("Unknown PIPELINE.")

In [ ]:
# @title 11. **Adelic Layer — ℚ_p Utilities & Prime Sweep** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Implements p-adic valuation/norm; builds composite “adelic balance”; sweeps primes (e.g., 2,3,5,7,11,…,137) for robustness.

from sympy import prime, factorint
from mpmath import mpf, power
import numpy as np
import matplotlib.pyplot as plt

# --- p-adic helpers ---
def padic_valuation(n, p):
    """Computes the p-adic valuation of an integer n."""
    if n == 0:
        return float('inf')
    valuation = 0
    n = abs(int(n)) # Ensure we are working with positive integers
    if n == 0: return float('inf')
    while n > 0 and n % p == 0:
        valuation += 1
        n //= p
    return valuation

def padic_norm(n, p):
    """Computes the p-adic norm |n|_p = p^(-v_p(n))."""
    if n == 0:
        return 0
    v = padic_valuation(n, p)
    return power(p, -v)

def padic_norm_from_energy(energy, scale=1e18):
    """
    Converts a real-valued energy to a scaled integer to compute a p-adic norm.
    This is a heuristic to bridge the continuous (real) and discrete (p-adic) domains.
    """
    # Heuristically convert a float energy to a large integer
    # The scale factor is crucial and problem-dependent
    n = int(energy * scale)
    if n == 0:
        return 0

    # For demonstration, we'll use a fixed prime p=2, but this could be varied.
    p = 2

    return padic_norm(n, p)

# --- Adelic Balance ---
# A conceptual model for combining real and p-adic measures.
# alpha_pad is a mixing parameter.
alpha_pad = mpf('0.1')

def adelic_balance(e_real, e_padic):
    """
    A simple model for the interplay between real and p-adic components.
    This is a toy model for the "adelic crystal" concept where both measures matter.
    """
    # Example: a weighted geometric mean, preserving a conceptual "total" value.
    # This is highly speculative and for model-testing purposes.
    e_real_new = power(e_real, 1 - alpha_pad)
    e_padic_new = power(e_padic, alpha_pad)

    return e_real_new, e_padic_new

# --- Prime Sweep ---
def get_primes_up_to(n):
    """Returns a list of prime numbers up to n."""
    primes = []
    i = 1
    while True:
        p = prime(i)
        if p <= n:
            primes.append(p)
            i += 1
        else:
            break
    return primes

def analyze_number_with_primes(n, primes):
    """
    Computes p-adic norms for a number n across a list of primes.
    """
    norms = [padic_norm(n, p) for p in primes]
    return norms

# --- Demonstration of Prime Sweep ---
# Example number to analyze. In the full model, this would be derived from a state variable.
test_number = 120  # (2^3 * 3 * 5)

# Primes to test against
primes_to_sweep = get_primes_up_to(20) # e.g., [2, 3, 5, 7, 11, 13, 17, 19]

# Calculate norms
norms = analyze_number_with_primes(test_number, primes_to_sweep)

# --- Report and Visualize ---
print(f"Analyzing integer: {test_number}")
print(f"Factors: {factorint(test_number)}")
print("p-adic norms |n|_p for primes up to 20:")
for p, norm in zip(primes_to_sweep, norms):
    print(f"  p={p:<3}: |{test_number}|_{p} = {float(norm):.4f}")

# Plotting the norms
plt.figure(figsize=(10, 5))
plt.bar([str(p) for p in primes_to_sweep], [float(n) for n in norms], color='skyblue')
plt.title(f'p-adic Norms of {test_number} for Various Primes')
plt.xlabel('Prime (p)')
plt.ylabel('p-adic Norm |n|_p')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Store adelic helpers and sweep results for later use
adelic_utils = {
    'padic_valuation': padic_valuation,
    'padic_norm': padic_norm,
    'padic_norm_from_energy': padic_norm_from_energy,
    'adelic_balance': adelic_balance,
    'alpha_pad': alpha_pad
}
prime_sweep_results = {
    'test_number': test_number,
    'primes': primes_to_sweep,
    'norms': norms
}

In [ ]:
# @title 12. **Breath Operator — Inhale/Exhale Rhythm** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Alternates phases to favour in-breath (0→C, even→C) and out-breath (C→odd, 1→2 work, 3→0 heat); non-local C stays mediated.

In [ ]:
# @title 13. **LENR Cycle Runner — Pump → Coherence → Squeeze → Fusion → Heat** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Executes the breath-driven cycle; integrates adelic scaling at node 3; detects “bursts” on phase-lock; logs heat/entropy.



def run_cycle(G_base: nx.DiGraph, steps=400, dt=mpf('0.05'), report_every=80):
    global phi_star
    energy = {n: mpf('0.5') for n in ALL}
    energy['1'] = mpf('1.0'); energy['5'] = mpf('1.0')   # start with E channels primed
    bursts = 0; entropy = mpf('0.0'); history = {'heat': [], 'entropy': [], 'bursts': [], 'phase': []}

    for k in range(steps):
        # Breath modulation (non-destructive per step)
        Gk, phase = apply_breath(G_base, k)

        # Simple conservative flow pass (rate * energy at source)
        for (u,v,data) in Gk.edges(data=True):
            flow = energy[u] * data['rate'] * dt
            if 'threshold' in data and flow < data['threshold']:
                continue
            energy[u] -= flow; energy[v] += flow

        # p-adic scaling at matter nodes (3,7)
        for m in ['3','7']:
            e_r = energy[m]
            e_p = padic_norm_from_energy(e_r)
            e_r, e_p = adelic_balance(e_r, e_p)
            energy[m] = e_r * power(e_p, alpha_pad)

        # Fusion bursts when C↔matter near phase lock (diagnostic, not physical prediction)
        # Use linter's phase (C fixed at 0) — here we proxy lock by small random jitter
        jitter = abs(np.sin(phi_star[idx['C']] - phi_star[idx['3']])) + \
                 abs(np.sin(phi_star[idx['C']] - phi_star[idx['7']]))
        if jitter < 0.2:
            # split burst between shells if available
            for m, sink in [('3','0'), ('7','4')]:
                burst = energy[m] * mpf('0.3')
                energy[m] -= burst
                energy[sink] += burst * mpf('0.9')
                entropy += burst * mpf('0.1')
            bursts += 1

        # Phase linter step (non-invasive)
        phi_star = phase_step(phi_star, float(dt))

        # Log
        history['heat'].append(float(energy['0'] + energy['4']))
        history['entropy'].append(float(entropy))
        history['bursts'].append(int(bursts))
        if k % report_every == 0:
            print(f"step {k:4d}  breath={phase:>4s}  bursts={bursts}  heat={history['heat'][-1]:.3f}  ",
                  phase_report(phi_star))
    return history

hist = run_cycle(G, steps=360)
print("\nFinal:", "bursts=", hist['bursts'][-1], "entropy=", hist['entropy'][-1])


In [ ]:
# @title 14. **Stable SI Stepper — Leapfrog + CFL + Joule Heating** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Uses leapfrog with CFL time step; computes SI energy, Joule power, entropy production; reports realistic field magnitudes.

In [ ]:
# @title 15. **Visualisations — Energy, EPR, Spectra, Phases** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Plots energy/frequency time-series, EPR bars (normal vs counterfactual vs low-T), residuals, phase diagnostics.

In [ ]:
# @title 16. **Red-Team Nulls & Ablations — Try to Break It** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Degree-preserving rewires; geometry swap (octahedron); no-adelic baseline; breath disabled; 2→1 during exhale only.

In [ ]:
# @title 17. **Preregistered Sweep — Seeds × Parameters × Outcomes** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Freezes thresholds; runs multiple seeds; aggregates pass/fail for quantised heat + mediation/EPR signatures.

In [ ]:
# @title 18. **Centroid Work vs Heat Ledger — Path Classification** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Labels forward “work-like” shortcuts (e.g., 0→1, 2→3) and backward “heat-like” fallbacks (1→0, 3→2); tallies contributions.

In [ ]:
# @title 19. **Star-Tetra Runner — Cross-Tetra Breath with Cubic Bridges** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Executes your S1↔S2 pathing rules; measures mediation across C, cross-domain hitting times, and burst propagation.

In [ ]:
# @title 20. **Centroid Locality Tests — Algorithmic Induction Around C** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Drives edge-based current scenarios near C; probes iterative/inductive dynamics; quantifies sensitivity to 109.47° bias.

In [ ]:
# @title 21. **Save & Export — Results, Seeds, Config** { display-mode: "form" }

# @markdown **What this does**

# @markdown - Stores metrics, plots, chosen primes, geometry flags, and RNG seeds for reproducibility; optional JSON/CSV export.

In [ ]:
# @title 22. **Appendix — Utilities & Helpers** { display-mode: "form" }

# @markdown **What this contains**

# @markdown - Small helpers (index maps, safe mat-ops, plotting styles), validation checks, and pretty printers.

# Task
Reorder the cells in the notebook "https://github.com/manifoldai/manifoldflow/blob/main/papers/unified_language/unified_language.ipynb" according to the following sequence, merging cells `7f7f6517` and `662659ce` into one cell, deleting cells `8eb7eb44` and `4fc07da8`, and noting that cell `vKkeHbZI-4fJ` should be parked for later: `qKR1ogm8760g`, `c3c07b46`, the merged cell (from `7f7f6517` and `662659ce`), `lVELWkflMr7b`, `-1pqe2HQ_n5m`, `bP96s6u7L2CW`.

## Merge cells

### Subtask:
Combine the content of cells `7f7f6517` and `662659ce` into a single code cell.


**Reasoning**:
Combine the code from the two specified cells into a single cell to merge the geometry definitions.



In [ ]:
# @title 2 & 3. **Geometry — Star Tetra (S1 + S2) with Cubic Bridges** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Defines S1 vertices {0,1,2,3,C}, allowed directed edges, self-loops, and “breath” semantics (work vs heat).
# @markdown - Adds S2 vertices {4,5,6,7} and inter-tetra “cubic” bridges; encodes the cross-tetra flow sequence you specified.


# Nodes: two tetrahedra share a single centroid C
S1 = ['0','1','2','3']          # sinks even / branches odd
S2 = ['4','5','6','7']
ALL = S1 + S2 + ['C']

G = nx.DiGraph()
G.add_nodes_from(ALL)

# Baseline rates (mpf for clean high-precision arithmetic)
r = lambda x: mpf(str(x))

# Core single-shell flows (work/heat + mediation)
core_edges_S1 = [
    ('C','1', {'rate': r(0.5)}),   # pump
    ('1','2', {'rate': r(0.8)}),   # E→M (work)
    ('2','C', {'rate': r(0.9)}),   # return to mediator
    ('C','3', {'rate': r(0.7)}),   # squeeze/fusion path
    ('3','0', {'rate': r(1.0)}),   # heat dump
    ('0','C', {'rate': r(0.3)}),   # reset
]

core_edges_S2 = [
    ('C','5', {'rate': r(0.5)}),
    ('5','6', {'rate': r(0.8)}),
    ('6','C', {'rate': r(0.9)}),
    ('C','7', {'rate': r(0.7)}),
    ('7','4', {'rate': r(1.0)}),
    ('4','C', {'rate': r(0.3)}),
]

# Cubic cross-edges (work bridge + heat dumps)
cross_edges = [
    ('3','6', {'rate': r(0.35)}),  # work bridge (coherent transfer S1→S2)
    ('7','2', {'rate': r(0.25)}),  # heat-biased back to S1 magnetic
    ('5','0', {'rate': r(0.20)}),  # heat dump assist into S1 sink
    ('1','6', {'rate': r(0.20)}),  # optional assist E→M across shells
]

# Self-loops (stability/linger)
self_loops = [(n, n, {'rate': r(0.4)}) for n in S1+S2]

G.add_edges_from(core_edges_S1 + core_edges_S2 + cross_edges + self_loops)
print("Graph ready. |V|=", len(G.nodes), "|E|=", len(G.edges))

**Reasoning**:
The previous command failed because the `networkx` library was not imported. I need to add the necessary import statement at the beginning of the combined cell.



In [ ]:
# @title 2 & 3. **Geometry — Star Tetra (S1 + S2) with Cubic Bridges** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Defines S1 vertices {0,1,2,3,C}, allowed directed edges, self-loops, and “breath” semantics (work vs heat).
# @markdown - Adds S2 vertices {4,5,6,7} and inter-tetra “cubic” bridges; encodes the cross-tetra flow sequence you specified.

import networkx as nx # Import networkx

# Nodes: two tetrahedra share a single centroid C
S1 = ['0','1','2','3']          # sinks even / branches odd
S2 = ['4','5','6','7']
ALL = S1 + S2 + ['C']

G = nx.DiGraph()
G.add_nodes_from(ALL)

# Baseline rates (mpf for clean high-precision arithmetic)
r = lambda x: mpf(str(x))

# Core single-shell flows (work/heat + mediation)
core_edges_S1 = [
    ('C','1', {'rate': r(0.5)}),   # pump
    ('1','2', {'rate': r(0.8)}),   # E→M (work)
    ('2','C', {'rate': r(0.9)}),   # return to mediator
    ('C','3', {'rate': r(0.7)}),   # squeeze/fusion path
    ('3','0', {'rate': r(1.0)}),   # heat dump
    ('0','C', {'rate': r(0.3)}),   # reset
]

core_edges_S2 = [
    ('C','5', {'rate': r(0.5)}),
    ('5','6', {'rate': r(0.8)}),
    ('6','C', {'rate': r(0.9)}),
    ('C','7', {'rate': r(0.7)}),
    ('7','4', {'rate': r(1.0)}),
    ('4','C', {'rate': r(0.3)}),
]

# Cubic cross-edges (work bridge + heat dumps)
cross_edges = [
    ('3','6', {'rate': r(0.35)}),  # work bridge (coherent transfer S1→S2)
    ('7','2', {'rate': r(0.25)}),  # heat-biased back to S1 magnetic
    ('5','0', {'rate': r(0.20)}),  # heat dump assist into S1 sink
    ('1','6', {'rate': r(0.20)}),  # optional assist E→M across shells
]

# Self-loops (stability/linger)
self_loops = [(n, n, {'rate': r(0.4)}) for n in S1+S2]

G.add_edges_from(core_edges_S1 + core_edges_S2 + cross_edges + self_loops)
print("Graph ready. |V|=", len(G.nodes), "|E|=", len(G.edges))

In [ ]:
# @title 2 & 3. **Geometry — Star Tetra (S1 + S2) with Cubic Bridges** { display-mode: "form" }
# @markdown **What this does**
# @markdown - Defines S1 vertices {0,1,2,3,C}, allowed directed edges, self-loops, and “breath” semantics (work vs heat).
# @markdown - Adds S2 vertices {4,5,6,7} and inter-tetra “cubic” bridges; encodes the cross-tetra flow sequence you specified.

import networkx as nx # Import networkx
from mpmath import mpf # Import mpf

# Nodes: two tetrahedra share a single centroid C
S1 = ['0','1','2','3']          # sinks even / branches odd
S2 = ['4','5','6','7']
ALL = S1 + S2 + ['C']

G = nx.DiGraph()
G.add_nodes_from(ALL)

# Baseline rates (mpf for clean high-precision arithmetic)
r = lambda x: mpf(str(x))

# Core single-shell flows (work/heat + mediation)
core_edges_S1 = [
    ('C','1', {'rate': r(0.5)}),   # pump
    ('1','2', {'rate': r(0.8)}),   # E→M (work)
    ('2','C', {'rate': r(0.9)}),   # return to mediator
    ('C','3', {'rate': r(0.7)}),   # squeeze/fusion path
    ('3','0', {'rate': r(1.0)}),   # heat dump
    ('0','C', {'rate': r(0.3)}),   # reset
]

core_edges_S2 = [
    ('C','5', {'rate': r(0.5)}),
    ('5','6', {'rate': r(0.8)}),
    ('6','C', {'rate': r(0.9)}),
    ('C','7', {'rate': r(0.7)}),
    ('7','4', {'rate': r(1.0)}),
    ('4','C', {'rate': r(0.3)}),
]

# Cubic cross-edges (work bridge + heat dumps)
cross_edges = [
    ('3','6', {'rate': r(0.35)}),  # work bridge (coherent transfer S1→S2)
    ('7','2', {'rate': r(0.25)}),  # heat-biased back to S1 magnetic
    ('5','0', {'rate': r(0.20)}),  # heat dump assist into S1 sink
    ('1','6', {'rate': r(0.20)}),  # optional assist E→M across shells
]

# Self-loops (stability/linger)
self_loops = [(n, n, {'rate': r(0.4)}) for n in S1+S2]

G.add_edges_from(core_edges_S1 + core_edges_S2 + cross_edges + self_loops)
print("Graph ready. |V|=", len(G.nodes), "|E|=", len(G.edges))